## Conclusion

The current Shiki integration fails due to:

1. **Version incompatibility** between stable lexical (0.33.1) and nightly @lexical/code-shiki (0.33.2-nightly)
2. **API mismatch** where internal code expects `getTheme()` method not provided by ShikiTokenizer
3. **Incomplete tokenizer wrapper** that doesn't properly bridge the interface gap

**Recommended immediate action:** Revert to stable Prism.js integration or upgrade all Lexical packages to matching nightly versions.

**Long-term:** Monitor the community implementation from the Discord discussion, which shows a comprehensive approach to Shiki integration with proper async loading and build system modifications.

The nightly @lexical/code-shiki package appears to be experimental and not ready for production use with stable Lexical versions.

### Long-term Solutions

1. **Wait for Stable Release:**
   - The Discord discussion shows extensive work on Shiki integration
   - Community member created a full implementation with backward compatibility
   - Official integration likely coming in future Lexical releases

2. **Implement Community Pattern:**
   - Follow the Discord discussion's architectural patterns
   - Separate plugin system for different highlighters
   - Async language loading with command pattern
   - Build system modifications for dynamic imports

3. **Alternative Approaches:**
   - Continue using Prism.js with existing stable integration
   - Consider lighter syntax highlighting libraries
   - Implement custom highlighting for specific languages needed

### Risk Assessment

**High Risk:**
- Nightly builds are experimental and may break
- Version mismatches cause runtime errors
- Minified code makes debugging difficult

**Medium Risk:**
- API changes between stable and nightly versions
- Build system complexity for dynamic imports
- Browser compatibility issues with advanced RegExp features

**Low Risk:**
- Using stable Prism.js integration
- Implementing basic syntax highlighting manually
- Waiting for official Shiki support

## Recommendations

### Immediate Solutions

1. **Version Alignment:**
   - Upgrade all @lexical packages to matching versions
   - Either use all stable (0.33.1) or all nightly builds
   - Avoid mixing stable and nightly versions

2. **Proper Tokenizer Implementation:**
   ```typescript
   class ShikiTokenizerWrapper implements Tokenizer {
     defaultLanguage = 'javascript';
     defaultTheme = 'one-light';
     
     getTheme(): string {
       return this.defaultTheme;
     }
     
     getLanguage(): string {
       return this.defaultLanguage;
     }
     
     $tokenize(codeNode: CodeNode, language?: string): LexicalNode[] {
       return ShikiTokenizer.$tokenize(codeNode, language);
     }
   }
   ```

3. **Interface Compatibility:**
   - Extend the wrapper with all methods the internal code expects
   - Properly bind method contexts
   - Handle async loading patterns from Discord implementation

In [ ]:
```typescript
// Expected Tokenizer interface from types
export interface Tokenizer {
    defaultLanguage: string;
    defaultTheme: string;
    $tokenize(codeNode: CodeNode, language?: string): LexicalNode[];
    // Missing: getTheme() method expected by internal code
}
```

**The gap:** The interface definition doesn't include `getTheme()` but the minified runtime code expects it.

In [ ]:
```typescript
// Current failing implementation
function CodeHighlightPlugin() {
  const [editor] = useLexicalComposerContext();

  useEffect(() => {
    // Create a wrapper that provides the missing getTheme method
    const wrappedTokenizer = {
      ...ShikiTokenizer,
      getTheme: () => ShikiTokenizer.defaultTheme || 'one-light',
      getLanguage: () => ShikiTokenizer.defaultLanguage || 'javascript',
    };
    
    // Cast to any to bypass version mismatch
    return registerCodeHighlighting(editor as any, wrappedTokenizer as any);
  }, [editor]);

  return null;
}
```

**Issues with this approach:**
1. Shallow object spread doesn't preserve method context
2. Version casting masks real compatibility issues
3. Missing integration with Shiki's async loading system

# Code Analysis

## Current Implementation Analysis

### What's Working

1. **Basic Setup:**
   - Lexical editor configured with CodeNode and CodeHighlightNode
   - @lexical/code-shiki package installed
   - ShikiTokenizer imported successfully

2. **Attempted Integration:**
   - Wrapper object created to provide missing `getTheme()` method
   - Plugin registers with editor successfully
   - Basic code block creation works (typing ```)

### What's Failing

1. **API Mismatch:**
   - Minified Shiki code calls `t2.getTheme()` but method doesn't exist
   - The wrapper approach doesn't properly integrate with internal tokenization
   - Version mismatch between core lexical (0.33.1) and code-shiki nightly (0.33.2)

2. **Tokenization Pipeline:**
   - Shiki tokenization happens in minified code that can't be easily debugged
   - Error occurs during space keypress after ``` input
   - Transformation from markdown shortcut to CodeNode triggers the error

### Discord Implementation Insights

Key patterns from the community implementation:

1. **Async Language Loading:**
   - Shiki requires dynamic imports for language grammars
   - Languages are loaded on-demand when first encountered
   - Uses command pattern with fire-and-forget async loading

2. **Plugin Architecture:**
   - Separate plugins for Prism vs Shiki to maintain backward compatibility
   - Node transforms handle the highlighting logic, not the nodes themselves
   - Editor commands coordinate async language loading

3. **Build Challenges:**
   - Build system needed modification to handle dynamic imports
   - Rollup configuration required changes from 'smallest' treeshaking preset
   - Browser compatibility issues with RegExp 'v' flag in oniguruma-to-es

### Version Compatibility Issues

From Discord and current analysis:

1. **Version Mismatch:**
   - Main lexical: v0.33.1
   - @lexical/code-shiki: v0.33.2-nightly.20250807.0 
   - This creates API incompatibilities

2. **Nightly Build Challenges:**
   - Using a nightly build with stable Lexical creates interface mismatches
   - The nightly build may have breaking changes not present in stable
   - Some APIs may be experimental or incomplete

3. **Interface Expectations:**
   - The minified code expects a `getTheme()` method on the tokenizer
   - Current `ShikiTokenizer` only provides `defaultTheme` and `defaultLanguage`
   - Missing wrapper methods that bridge the API gap

## Key Findings from Discord Discussion

### Background Context

The Discord discussion reveals an extensive effort by a community member (inkie) to replace Prism.js with Shiki in the Lexical editor. Key insights:

1. **Shiki vs Prism Architecture Difference:**
   - Prism provides structured token types (operator, comment, property, etc.)
   - Shiki applies themes during tokenization and inlines styles as properties (color, etc.)
   - Lexical expects structural token types, but Shiki only provides styled tokens

2. **The Integration Challenge:**
   - Shiki doesn't expose the structural token types that Lexical's CodeHighlightNode expects
   - Shiki themes are TypeScript files called during tokenization, not CSS files
   - Theme switching requires different integration patterns than Prism

# Lexical Shiki Integration Analysis

This notebook analyzes the Shiki integration issues in the Lexical editor, based on the Discord discussion from the Lexical Discord server and investigation of the current codebase.

## Problem Summary

The current implementation fails with:
```
TypeError: t2.getTheme is not a function
```

This error occurs when pressing space after typing ``` (three backticks) to create a code block.